**Fully Connected Neural Network – Titanic Survival Prediction**

This notebook implements a **Fully Connected Neural Network (FNN)** using Keras to predict Titanic passenger survival based on structured tabular data. It covers full data preprocessing, model building, training with regularization, and evaluation.

**Objective**
Build a binary classifier using deep learning to predict whether a passenger survived the Titanic disaster using structured features (e.g., age, gender, fare, class, etc.).

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder

**1. Data Preprocessing**
- Dropped irrelevant columns: `PassengerId`, `Name`, `Ticket`, `Cabin`
- Handled missing values:
  - `Age` filled with median
  - `Embarked` filled with mode
- Encoded categorical variables:
  - `Sex`: female → 0, male → 1
  - `Embarked`: C → 0, Q → 1, S → 2 (LabelEncoded)
- Split data into train/test sets (80/20)
- Scaled numerical features using `StandardScaler`

In [10]:
df = pd.read_csv('/kaggle/input/titanic-dataset/Titanic-Dataset.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [12]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [13]:
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

/tmp/ipykernel_36/2359842934.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
/tmp/ipykernel_36/2359842934.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [14]:
df['Sex'] = df['Sex'].map({'female': 0, 'male': 1})


In [16]:
le = LabelEncoder()
df['Embarked'] = le.fit_transform(df['Embarked'])

In [17]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [18]:
# Split features and target
X = df.drop('Survived', axis=1)
y = df['Survived']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
X_train[:10]

array([[-1.61413602,  0.7243102 ,  1.25364106, -0.47072241, -0.47934164,
        -0.07868358,  0.5635246 ],
       [-0.40055118,  0.7243102 , -0.47728355, -0.47072241, -0.47934164,
        -0.37714494,  0.5635246 ],
       [ 0.81303367,  0.7243102 ,  0.21508629, -0.47072241, -0.47934164,
        -0.47486697,  0.5635246 ],
       [ 0.81303367,  0.7243102 , -0.24649361,  0.37992316, -0.47934164,
        -0.47623026,  0.5635246 ],
       [ 0.81303367, -1.38062393, -1.78509326,  2.93185988,  2.04874166,
        -0.02524937,  0.5635246 ],
       [-1.61413602,  0.7243102 , -0.40035357, -0.47072241,  0.78470001,
         4.13868709, -2.02505292],
       [-1.61413602,  0.7243102 ,  1.21517607, -0.47072241, -0.47934164,
        -0.11623195,  0.5635246 ],
       [-0.40055118,  0.7243102 , -0.01570366,  0.37992316, -0.47934164,
        -0.09368752, -2.02505292],
       [ 0.81303367,  0.7243102 , -0.09263364, -0.47072241, -0.47934164,
        -0.47542923,  0.5635246 ],
       [-1.61413602,  0.7243

In [21]:
X_test

array([[ 0.81303367,  0.7243102 , -0.09263364, ...,  0.78470001,
        -0.33390078, -2.02505292],
       [-0.40055118,  0.7243102 ,  0.13815631, ..., -0.47934164,
        -0.42528387,  0.5635246 ],
       [ 0.81303367,  0.7243102 , -0.7080735 , ..., -0.47934164,
        -0.47486697,  0.5635246 ],
       ...,
       [ 0.81303367, -1.38062393,  0.67666619, ...,  5.8408666 ,
        -0.02308312,  0.5635246 ],
       [-0.40055118, -1.38062393, -0.93886345, ..., -0.47934164,
        -0.42528387,  0.5635246 ],
       [ 0.81303367, -1.38062393, -1.93895323, ...,  0.78470001,
        -0.30589933,  0.5635246 ]])

**2. Model Architecture (FNN)**

Input layer: Size = number of features

Hidden layers: ReLU activations

Output layer: Sigmoid (binary classification)

In [22]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary output
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-08-04 07:26:52.804475: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


**3. Training and Evaluation**

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=16, verbose=0)

In [26]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8112 - loss: 0.4553 
Test Accuracy: 0.8212


**4. Classfication Report**

In [27]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
              precision    recall  f1-score   support

           0       0.83      0.88      0.85       105
           1       0.81      0.74      0.77        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179



**5. Training After Regularization by Adding dropout layers**

In [28]:
from tensorflow.keras.layers import Dropout

model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=16, verbose=0)

In [30]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8111 - loss: 0.4172 
Test Accuracy: 0.8101


**6. Callbacks**

Added EarlyStopping to monitor val_loss with patience=5

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=16,
    callbacks=[early_stop],
    verbose=0
)

In [32]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8111 - loss: 0.4157 
Test Accuracy: 0.8101


**7. Training after adding more layers to improve the accuracy**

In [44]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary output
])

In [45]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=16,
    callbacks=[early_stop],
    verbose=0
)

In [46]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8279 - loss: 0.4192 
Test Accuracy: 0.8212


**Key Learnings**

FNNs can handle structured data but require good preprocessing

Simple models often perform competitively with deep ones on small data

Regularization (dropout, early stopping) is essential for generalization

Proper encoding and scaling drastically improve performance